In [1]:
import pandas as pd
from pandas import isnull
from nltk.corpus import brown
from nltk.tokenize import TreebankWordDetokenizer, word_tokenize
from random import sample, shuffle


def contains_prof(text, profanities) :
    tokenized_text = word_tokenize(text)
    return (not not set(tokenized_text).intersection(set(profanities)))

def ascii_range(text) :
    for c in text :
        if ord(c) not in range(128) : return False
    return True

In [2]:
prof_df_original = pd.read_csv("datasets/profanity_en.csv",sep=',')
prof_list = []
labels = ['canonical_form_1', 'canonical_form_2', 'canonical_form_3']
for _, row in prof_df_original.iterrows():
    canons = [i for i in list(row[labels]) if not isnull(i)]
    prof_list += canons
prof_list = list(set(prof_list))

df = pd.read_csv("datasets/reddit_wsb.csv",sep=',')
df.dropna(inplace=True)
df = df[['title','body']]
print(len(df))

24738


In [3]:
hate_sents = []

for idx, row in df.iterrows() :
    text = row['title']
    if contains_prof(text, prof_list) and len(text) <= 256 and ascii_range(text):
        hate_sents.append(text)

print(len(hate_sents))

candidate_sents = [s for s in brown.sents() if len(s) <= 50]

sents = sample(candidate_sents, len(hate_sents))
tk = TreebankWordDetokenizer()
nothate_sents = [tk.detokenize(s) for s in sents]

428


In [11]:
new_labels = ["hate" for i in range(len(hate_sents))] + ["nothate" for i in range(len(nothate_sents))]
new_sents = hate_sents + nothate_sents

new_list = list(zip(new_sents, new_labels))
shuffle(new_list)
new_df = pd.DataFrame(new_list, columns=['text', 'label'])
new_df.to_csv("augmented_data/Dataset_new_{i}.csv".format(i=len(new_df)), sep=',')